In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree

import requests
from lxml import html


## Функции

In [2]:
def get_div(tree, xpath_format, div_mode=True):
    list_divs = []


    # Выбор элемента
    selected_element = tree.xpath(xpath_format)


    for div in selected_element:
        # Получение HTML-разметки только для выбранного элемента
        html_representation = html.tostring(div, encoding='unicode', method='html')

        list_divs.append(html_representation)
    return list_divs


# Main

In [4]:
dict_info = {}

list_sites  = ['https://victorymuseum.ru/playbill/']

list_xpaths = ['//div[@class="oblique-items"]//div[@class="item"]']


for site, xpath  in zip(list_sites, list_xpaths):
    response = requests.get(site)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        
        
        target_obj = get_div(tree, xpath)  
        dict_info[site] = target_obj
        
        
    else:
        print(f'Ошибка статуса {response.status_code} для {sites}')

In [11]:
test = dict_info[site][0]

In [22]:
df = pd.DataFrame(dict_info)

In [29]:
df['https://victorymuseum.ru/playbill/'].str.extract('>([A-Za-zА-Яа-я ,0-9]+)')

,0
0,События
1,События
2,События
3,События
4,События
5,Выставки
6,События
7,События
8,События


In [36]:
df['https://victorymuseum.ru/playbill/'][1]

'<div class="item" id="bx_3218110189_255458" data-id="28">\n\t\t<a href="/playbill/events/80-let-tegeranskoy-konferentsii/" class="img-box">\n\t\t\t\t\t\t<div class="img-item-bg lazyload" data-bgset="/upload/resize_cache/iblock/07c/9v78mgpjza3b9a40k35fpfqc39kap9z7/550_380_2/IMG_6693.jpg"></div>\n\t\t</a>\n\t\t<div class="text-box">\n\t\t\t\t\t\t\t<div class="w100">\n\t\t\t\t\t<a class="button mini" href="/playbill/events/">События</a>\n\t\t\t\t</div>\n\t\t\t\n\t\t\t<a class="name" href="/playbill/events/80-let-tegeranskoy-konferentsii/">80 лет Тегеранской конференции</a>\n\t\t\t\t\t\t\t<div class="text">К юбилею\xa0конференции «большой тройки» Музей Победы подготовил онлайн-программу.</div>\n\t\t\t\t\t\t\n\t\t\t\n\t\t</div>\n\t\t<div class="bottom_box">\n\t\t\t\t\t<div class="date ico-ba">28.11.2023</div>\n\t\t\t\t\t\t\t\t\t\n\t\t\t\n\t\t\t\t\t\t<div class="ticker-ref">\n\t\t\t\t\t\t\t</div>\n\t\t\t\n\t\t\t\t\t</div>\n\t</div>\n\t\n\t\n\n\t\n\t\t'

In [38]:
df['https://victorymuseum.ru/playbill/'].str.extractall('>([A-Za-zА-Яа-я ,0-9-.\\]+)')

error: unterminated character set at position 2

In [18]:
'>[A-Za-zА-Яа-я ,0-9]+'

'>[A-Za-zА-Яа-я ,0-9]+'

In [17]:
print(test)

<div class="item" id="bx_3218110189_251266" data-id="28">
		<a href="/playbill/events/den-neizvestnogo-soldata-/" class="img-box">
						<div class="img-item-bg lazyload" data-bgset="/upload/resize_cache/iblock/799/lvh3xijw14j6r5o2y05bdthkse7khy7f/550_380_2/muzej_1.jpeg"></div>
		</a>
		<div class="text-box">
							<div class="w100">
					<a class="button mini" href="/playbill/events/">События</a>
				</div>
			
			<a class="name" href="/playbill/events/den-neizvestnogo-soldata-/">День неизвестного солдата </a>
							<div class="text">Памятная дата в России, учреждена указом Президента РФ в 2014 году</div>
						
			
		</div>
		<div class="bottom_box">
					<div class="date ico-ba">03.12.2023</div>
									
			
						<div class="ticker-ref">
							</div>
			
					</div>
	</div>
	
	

	
		


<div class="item" id="bx_3218110189_251266" data-id="28">\n\t\t<a href="/playbill/events/den-neizvestnogo-soldata-/" class="img-box">\n\t\t\t\t\t\t
<div class="img-item-bg lazyload" data-bgset="/upload/resize_cache/iblock/799/lvh3xijw14j6r5o2y05bdthkse7khy7f/550_380_2/muzej_1.jpeg"></div>
\n\t\t</a>\n\t\t<div class="text-box">\n\t\t\t\t\t\t\t<div class="w100">\n\t\t\t\t\t
<a class="button mini" href="/playbill/events/">События</a>\n\t\t\t\t</div>\n\t\t\t\n\t\t\t<a class="name" href="/playbill/events/den-neizvestnogo-soldata-/">День неизвестного солдата </a>\n\t\t\t\t\t\t\t<div class="text">Памятная дата в России, учреждена указом Президента РФ в 2014 году</div>\n\t\t\t\t\t\t\n\t\t\t\n\t\t</div>\n\t\t<div class="bottom_box">\n\t\t\t\t\t<div class="date ico-ba">03.12.2023</div>\n\t\t\t\t\t\t\t\t\t\n\t\t\t\n\t\t\t\t\t\t<div class="ticker-ref">\n\t\t\t\t\t\t\t</div>\n\t\t\t\n\t\t\t\t\t</div>\n\t</div>\n\t\n\t\n\n\t\n\t\t

In [14]:
print('''\n\t\t\n\t\t\t\t\t\t
\n\t\t\n\t\t
\n\t\t\t\t\t\t\t
\n\t\t\t\t\tСобытия\n\t\t\t\t
\n\t\t\t\n\t\t\tДень неизвестного солдата \n\t\t\t\t\t\t\t
Памятная дата в России, учреждена указом Президента РФ в 2014 году
\n\t\t\t\t\t\t\n\t\t\t\n\t\t
\n\t\t
\n\t\t\t\t\t
03.12.2023
\n\t\t\t\t\t\t\t\t\t\n\t\t\t\n\t\t\t\t\t\t
\n\t\t\t\t\t\t\t
\n\t\t\t\n\t\t\t\t\t
\n\t
\n\t\n\t\n\n\t\n\t\tau''')


		
						

		
		

							

					События
				

			
			День неизвестного солдата 
							
Памятная дата в России, учреждена указом Президента РФ в 2014 году

						
			
		

		

					
03.12.2023

									
			
						

							

			
					

	

	
	

	
		au


<div class="item" id="bx_3218110189_251266" data-id="28">\n\t\t<a href="/playbill/events/den-neizvestnogo-soldata-/" class="img-box">\n\t\t\t\t\t\t<div class="img-item-bg lazyload" data-bgset="/upload/resize_cache/iblock/799/lvh3xijw14j6r5o2y05bdthkse7khy7f/550_380_2/muzej_1.jpeg"></div>\n\t\t</a>\n\t\t<div class="text-box">\n\t\t\t\t\t\t\t<div class="w100">\n\t\t\t\t\t<a class="button mini" href="/playbill/events/">События</a>\n\t\t\t\t</div>\n\t\t\t\n\t\t\t<a class="name" href="/playbill/events/den-neizvestnogo-soldata-/">День неизвестного солдата </a>\n\t\t\t\t\t\t\t<div class="text">Памятная дата в России, учреждена указом Президента РФ в 2014 году</div>\n\t\t\t\t\t\t\n\t\t\t\n\t\t</div>\n\t\t<div class="bottom_box">\n\t\t\t\t\t<div class="date ico-ba">03.12.2023</div>\n\t\t\t\t\t\t\t\t\t\n\t\t\t\n\t\t\t\t\t\t<div class="ticker-ref">\n\t\t\t\t\t\t\t</div>\n\t\t\t\n\t\t\t\t\t</div>\n\t</div>\n\t\n\t\n\n\t\n\t\t

In [ ]:
dict_info = {}

list_sites  = ['https://victorymuseum.ru/playbill/']

list_xpaths = ['//div[@class="oblique-items"]//div[@class="item"]']


for site, xpath  in zip(list_sites, list_xpaths):
    response = requests.get(site)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        
        
        target_obj = get_div(tree, xpath)  
        dict_info[site] = target_obj
        
        
    else:
        print(f'Ошибка статуса {response.status_code} для {sites}')

In [8]:
target_obj[0]

'<div class="item" id="bx_3218110189_251266" data-id="28">\n\t\t<a href="/playbill/events/den-neizvestnogo-soldata-/" class="img-box">\n\t\t\t\t\t\t<div class="img-item-bg lazyload" data-bgset="/upload/resize_cache/iblock/799/lvh3xijw14j6r5o2y05bdthkse7khy7f/550_380_2/muzej_1.jpeg"></div>\n\t\t</a>\n\t\t<div class="text-box">\n\t\t\t\t\t\t\t<div class="w100">\n\t\t\t\t\t<a class="button mini" href="/playbill/events/">События</a>\n\t\t\t\t</div>\n\t\t\t\n\t\t\t<a class="name" href="/playbill/events/den-neizvestnogo-soldata-/">День неизвестного солдата </a>\n\t\t\t\t\t\t\t<div class="text">Памятная дата в России, учреждена указом Президента РФ в 2014 году</div>\n\t\t\t\t\t\t\n\t\t\t\n\t\t</div>\n\t\t<div class="bottom_box">\n\t\t\t\t\t<div class="date ico-ba">03.12.2023</div>\n\t\t\t\t\t\t\t\t\t\n\t\t\t\n\t\t\t\t\t\t<div class="ticker-ref">\n\t\t\t\t\t\t\t</div>\n\t\t\t\n\t\t\t\t\t</div>\n\t</div>\n\t\n\t\n\n\t\n\t\t'

<div class="item" id="bx_3218110189_251266" data-id="28">\n\t\t<a href="/playbill/events/den-neizvestnogo-soldata-/" class="img-box">\n\t\t\t\t\t\t<div class="img-item-bg lazyload" data-bgset="/upload/resize_cache/iblock/799/lvh3xijw14j6r5o2y05bdthkse7khy7f/550_380_2/muzej_1.jpeg"></div>\n\t\t</a>\n\t\t<div class="text-box">\n\t\t\t\t\t\t\t<div class="w100">\n\t\t\t\t\t<a class="button mini" href="/playbill/events/">События</a>\n\t\t\t\t</div>\n\t\t\t\n\t\t\t<a class="name" href="/playbill/events/den-neizvestnogo-soldata-/">День неизвестного солдата </a>\n\t\t\t\t\t\t\t<div class="text">Памятная дата в России, учреждена указом Президента РФ в 2014 году</div>\n\t\t\t\t\t\t\n\t\t\t\n\t\t</div>\n\t\t<div class="bottom_box">\n\t\t\t\t\t<div class="date ico-ba">03.12.2023</div>\n\t\t\t\t\t\t\t\t\t\n\t\t\t\n\t\t\t\t\t\t<div class="ticker-ref">\n\t\t\t\t\t\t\t</div>\n\t\t\t\n\t\t\t\t\t</div>\n\t</div>\n\t\n\t\n\n\t\n\t\t

In [ ]:
dict_info = {}

list_sites    = ['https://victorymuseum.ru/playbill/', 'https://stalingrad-battle.ru/about/m-news/', 
                 'https://blokadamus.ru/музей-обороны-и-блокады-ленинграда/события/']

xpaths_format = ["//div[@data-id='28']", '//*[@class="item"]', '//*[@class="elementor-post__card"]', ]

for site, xpath_format in zip(list_sites, xpaths_format):
    response = requests.get(site)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        dict_info[site] = get_div(tree, xpath_format)
    else:
        print(f'Ошибка статуса {response.status_code} для {sites}')

## div = False

In [ ]:
dict_info = {}

list_sites    = ['http://www.cmaf.ru/']


xpaths_format = ['//div[@class="news-list"]//a[@style="font-size: 15px; font-weight: bold; margin: 0;"]']
for site, xpath_format in zip(list_sites, xpaths_format):
    response = requests.get(site)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        dict_info[site] = get_div(tree, xpath_format, div_mode=False)
    else:
        print(f'Ошибка статуса {response.status_code} для {sites}')

In [ ]:
dict_info = {}

list_sites    = ['https://1418museum.ru/news/']
xpaths_format = ['//span[@class="newsItem__title title title--small"]']




for site, xpath_format in zip(list_sites, xpaths_format):
    response = requests.get(site)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        dict_info[site] = get_div(tree, xpath_format, div_mode=False)
    else:
        print(f'Ошибка статуса {response.status_code} для {sites}')

In [ ]:
dict_info = {}

list_sites    = ['https://victorymuseum.ru/playbill/']
xpaths_tittle = ["//a[@class='name']"]


xpath_format = ["//div[@class='text']"]




for site, xpath_format in zip(list_sites, xpaths_format):
    response = requests.get(site)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        dict_info[site] = get_div(tree, xpath_format, div_mode=False)
    else:
        print(f'Ошибка статуса {response.status_code} для {sites}')

In [ ]:
dict_info[site]

In [ ]:
dict_info[site]

In [ ]:
https://victorymuseum.ru/playbill/

In [ ]:
, div_mode=False

In [ ]:
class="newsItem__description text"

In [ ]:
dict_info

In [ ]:
print(response.text)

In [ ]:
tree = html.fromstring(response.text)

In [ ]:
tree

In [ ]:
dict_info

In [ ]:
//span[@class="newsItem__title title title--small"]

In [ ]:
selected_element[0]

In [ ]:
len(selected_element)

In [ ]:
    


    for div in selected_element:

        # # Создаем временный элемент и копируем содержимое выбранного элемента
        temp_element = html.Element("div")
        temp_element.extend(div)

        # Получение HTML-разметки только для выбранного элемента
        html_representation = html.tostring(temp_element, encoding='unicode', method='html')

        list_divs.append(html_representation)
    return list_divs

In [ ]:
tree

In [ ]:
xpath_format

In [ ]:
tree

In [ ]:
dict_info

In [ ]:
for site, xpath_format in zip(list_sites, xpaths_format):
    response = requests.get(site)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        print('ok')

In [ ]:
soup.find_all

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

# Найти первый элемент <p> с классом "news-item"
first_news_item = soup.find_all('p', class_='news-item')

# Вывести содержимое найденного элемента
print(first_news_item)

In [ ]:
first_news_item[0]

In [ ]:
first_news_item[1]

In [ ]:
first_news_item[2]

In [ ]:
temp_element = html.Element("p")

In [ ]:
temp_element.extend(temp_element)

In [ ]:
print(html.tostring(temp_element, encoding='unicode', method='html'))

In [ ]:
dict_info

In [ ]:
selected_element = 

In [ ]:
selected_element[3].text_content()

In [ ]:
print(div)

In [ ]:
div = selected_element[0]

temp_element = html.Element("a")
# temp_element.extend(elem)

# # Получение HTML-разметки только для выбранного элемента
# html_representation = html.tostring(temp_element, encoding='unicode', method='html')

# print(html_representation)

In [ ]:
html_representation = html.tostring(temp_element, encoding='unicode', method='html')

In [ ]:
html_representation

In [ ]:
стоп

## Как можно парсить

In [ ]:
cnjg

In [ ]:
# Парсинг HTML
tree = html.fromstring(response.text)

# Выбор элемента
selected_element = tree.xpath("//div[@data-id='28']")[0]

# # Создаем временный элемент и копируем содержимое выбранного элемента
temp_element = html.Element("div")

temp_element.extend(selected_element)

# Получение HTML-разметки только для выбранного элемента
html_representation = html.tostring(temp_element, encoding='unicode', method='html')

print(html_representation)


In [ ]:
# Парсинг HTML
tree = html.fromstring(response.text)

# Выбор элемента
selected_element = tree.xpath("//div[@data-id='28']")[0]

# # Создаем временный элемент и копируем содержимое выбранного элемента
temp_element = html.Element("div")
temp_element.extend(selected_element)

# Получение HTML-разметки только для выбранного элемента
html_representation = html.tostring(temp_element, encoding='unicode', method='html')

print(html_representation)